In [ ]:
#IMPORT THE NECESSARY LIBRARIES
from flask import Flask, render_template, request, redirect, url_for
from pymongo import MongoClient
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

####MONGODB SETUP####
app = Flask(__name__)

client = MongoClient('mongodb://localhost:27017/')
db = client['user_data_db']
collection = db['users']

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        user_data = {
            'age': request.form['age'],
            'gender': request.form['gender'],
            'total_income': request.form['total_income'],
            'expenses': {
                'utilities': request.form.get('utilities', 0),
                'entertainment': request.form.get('entertainment', 0),
                'school_fees': request.form.get('school_fees', 0),
                'shopping': request.form.get('shopping', 0),
                'healthcare': request.form.get('healthcare', 0)
            }
        }
        # Insert data into MongoDB
        collection.insert_one(user_data)
        return redirect(url_for('index'))
    
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)


# MongoDB setup
client = MongoClient('mongodb://localhost:27017/')
db = client['user_data_db']
collection = db['users']

# Define the User class
class User:
    def __init__(self, age, gender, total_income, expenses):
        self.age = age
        self.gender = gender
        self.total_income = total_income
        self.expenses = expenses

# Retrieve data from MongoDB and write to CSV
with open('user_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['age', 'gender', 'total_income', 'utilities', 'entertainment', 'school_fees', 'shopping', 'healthcare']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for record in collection.find():
        user = User(
            age=record['age'],
            gender=record['gender'],
            total_income=record['total_income'],
            expenses=record['expenses']
        )
        writer.writerow({
            'age': user.age,
            'gender': user.gender,
            'total_income': user.total_income,
            'utilities': user.expenses.get('utilities', 0),
            'entertainment': user.expenses.get('entertainment', 0),
            'school_fees': user.expenses.get('school_fees', 0),
            'shopping': user.expenses.get('shopping', 0),
            'healthcare': user.expenses.get('healthcare', 0)
        })

####DATA VISUALIZATION####

# Load data
df = pd.read_csv('user_data.csv')

# Visualization 1: Ages with the Highest Income
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='age', y='total_income', ci=None)
plt.title('Ages with the Highest Income')
plt.xlabel('Age')
plt.ylabel('Total Income')
plt.savefig('ages_highest_income.png')
plt.show()

# Visualization 2: Gender Distribution Across Spending Categories
expense_columns = ['utilities', 'entertainment', 'school_fees', 'shopping', 'healthcare']
df_expenses = df.melt(id_vars='gender', value_vars=expense_columns, var_name='Expense Category', value_name='Amount')

plt.figure(figsize=(10, 6))
sns.boxplot(data=df_expenses, x='Expense Category', y='Amount', hue='gender')
plt.title('Gender Distribution Across Spending Categories')
plt.xlabel('Expense Category')
plt.ylabel('Amount Spent')
plt.savefig('gender_distribution_spending.png')
plt.show()


#####HTML TEMPLATE TO COLLECT USER DATA#####
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>User Data Collection</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
</head>
<body>
    <h1>Enter Your Details</h1>
    <form method="POST">
        <label for="age">Age:</label><br>
        <input type="number" id="age" name="age" required><br><br>
        
        <label for="gender">Gender:</label><br>
        <select id="gender" name="gender" required>
            <option value="Male">Male</option>
            <option value="Female">Female</option>
            <option value="Other">Other</option>
        </select><br><br>
        
        <label for="total_income">Total Income:</label><br>
        <input type="number" id="total_income" name="total_income" required><br><br>
        
        <h3>Expenses</h3>
        <label for="utilities">Utilities:</label><br>
        <input type="number" id="utilities" name="utilities"><br><br>

        <label for="entertainment">Entertainment:</label><br>
        <input type="number" id="entertainment" name="entertainment"><br><br>

        <label for="school_fees">School Fees:</label><br>
        <input type="number" id="school_fees" name="school_fees"><br><br>

        <label for="shopping">Shopping:</label><br>
        <input type="number" id="shopping" name="shopping"><br><br>

        <label for="healthcare">Healthcare:</label><br>
        <input type="number" id="healthcare" name="healthcare"><br><br>

        <input type="submit" value="Submit">
    </form>
</body>
</html>
